# Initialization

In [2]:
# Data scraping from uderstat, dataframe prep and csv output

In [3]:
import requests
from bs4 import BeautifulSoup

import json
import pandas as pd

In [4]:
player_url = "https://understat.com/player/204"
dict_of_script_positions = {1:'aggregated',2:'radar',3:'shots',4:'matches'}
reqs = requests.get(player_url)
soup = BeautifulSoup(reqs.content,'lxml')
scripts = soup.find_all('script')

In [5]:
for pos in  dict_of_script_positions:
    strings = scripts[pos].string
    ind_start = strings.index("('")+2 
    ind_end = strings.index("')") 
    json_data = strings[ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')
    data = json.loads(json_data)
    if pos == 1:
        vars()["df_" +dict_of_script_positions[pos] + "_season"] = pd.DataFrame(data['season'])        
        vars()["df_" +dict_of_script_positions[pos] + "_shotTypes"] = pd.DataFrame()
        for x in pd.DataFrame(data['shotTypes']).T.index:
            vars()["df_" +dict_of_script_positions[pos] + "_shotTypes"] = vars()["df_" +dict_of_script_positions[pos] + "_shotTypes"].append(pd.DataFrame(data['shotTypes']).T.loc[x].apply(pd.Series).set_index(['season', 'shotTypes']))
    elif pos == 2:
        vars()["df_" +dict_of_script_positions[pos]] = pd.DataFrame()
        for x in pd.DataFrame(data).T.index:
            temp_df = pd.DataFrame(data).T.loc[x].apply(pd.Series)
            temp_df['position'] = x
            vars()["df_" +dict_of_script_positions[pos]] = vars()["df_" +dict_of_script_positions[pos]].append(temp_df)
           
    else:
        vars()["df_" +dict_of_script_positions[pos]] = pd.DataFrame(data)
    print(["df_" +dict_of_script_positions[pos]])

['df_aggregated']
['df_radar']
['df_shots']
['df_matches']


In [6]:
%whos DataFrame

Variable                  Type         Data/Info
------------------------------------------------
df_aggregated_season      DataFrame      position games goals sh<...>198   14.59677966311574  
df_aggregated_shotTypes   DataFrame                     goals sh<...>                NaN NaN  
df_matches                DataFrame        goals shots          <...>\n[253 rows x 20 columns]
df_radar                  DataFrame                     min     <...>008153  0.002759      Sub
df_shots                  DataFrame             id minute       <...>\n[289 rows x 20 columns]
temp_df                   DataFrame                     min     <...>008153  0.002759      Sub


In [7]:
# output data through excel writer
with pd.ExcelWriter("GranitXhakaOutputData.xlsx") as writer:
    df_aggregated_season.to_excel(writer, sheet_name="Seasons", index=False)
    df_aggregated_shotTypes.to_excel(writer, sheet_name="ShotTypes", index=False)
    df_matches.to_excel(writer, sheet_name="Matches", index=False)
    df_radar.to_excel(writer, sheet_name="Radar", index=False)
    df_shots.to_excel(writer, sheet_name="Shots", index=False)